In [1]:
import tensorflow as tf
from keras.layers import Dense
import json
import numpy as np
import librosa
import pandas as pd
from model_utils import save_model
import glob, os

In [2]:
from tensorflow import keras

In [50]:
import glob
import soundfile

In [51]:
y, sr = librosa.load('/Users/cooky/Desktop/data/02-03.wav')
y

array([ 6.5707300e-06,  3.4268778e-06, -1.4703763e-06, ...,
        1.3547251e-06,  1.1079610e-06,  9.1189868e-06], dtype=float32)

In [31]:
X_input = []
x_label = []
y_output = []
max_len = 1 * 44100

for i in range(77):
# for i in range(3):
    x, sr_x = librosa.load('/Users/cooky/Desktop/data/'+str(i).zfill(2)+'.wav', mono=False)
    x = x[0]
    chg = glob.glob('/Users/cooky/Desktop/data/'+str(i).zfill(2)+'-*.wav')[0]
    y, sr_y = librosa.load(chg, mono=False)
    y = y[0]
    if (len(y) != len(x)) or (sr_x != sr_y) or (len(x) < max_len):
        print('Err ' + str(i))
    else:
        print(str(i))
        for j in range(0, len(x), max_len):
            x_chunk = x[j:j+max_len]
            y_chunk = y[j:j+max_len]
            if len(x_chunk) == max_len:
                X_input.append(x_chunk)
                y_output.append(y_chunk)
        # np.append(y_output, y[:tmp], axis=0)
                tmpp=[]
                for _ in range(max_len):
                    tmpp.append(int(chg[-6:-4]))
                x_label.append(tmpp)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
Err 28
Err 29
30
31
32
33
34
Err 35
36
Err 37
38
39
40
Err 41
42
Err 43
Err 44
45
Err 46
Err 47
Err 48
49
50
51
Err 52
53
Err 54
55
Err 56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76


In [32]:
X_input = np.array(X_input)
y_output = np.array(y_output)
x_label = np.array(x_label)

In [33]:
X_input_res = np.reshape(X_input, (X_input.shape[0],X_input.shape[1],1))
x_label_res = np.reshape(x_label, (x_label.shape[0],x_label.shape[1],1))
X_inputt = np.concatenate((X_input_res, x_label_res), axis=2)

In [34]:
X_inputt.shape

(1731, 44100, 2)

In [35]:
y_output.shape

(1731, 44100)

In [36]:
model = keras.Sequential(name='model')
model.add(keras.layers.InputLayer(input_shape=(None,2)))
# model.add(keras.layers.Dense(4,name='D1', activation='relu'))
model.add(keras.layers.LSTM(8, name='L'))
# model.add(keras.layers.LSTM(16, name='L'))
model.add(keras.layers.Dense(16,name='D3'))
model.add(keras.layers.Dense(1, name='fc'))

In [37]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 L (LSTM)                    (None, 8)                 352       
                                                                 
 D3 (Dense)                  (None, 16)                144       
                                                                 
 fc (Dense)                  (None, 1)                 17        
                                                                 
Total params: 513
Trainable params: 513
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [39]:
with tf.device('/GPU:0'):
    model.fit(x=X_inputt, y=y_output, validation_split=0.1, batch_size=16, epochs=10)

Epoch 1/10


KeyboardInterrupt: 

In [46]:
X_inputttt = X_inputtt.reshape(X_inputtt.shape[0],1,2)

In [44]:
X_inputtt.shape

(10671370, 2)

In [21]:
y = np.array([1,2,3])

In [23]:
y[2:4]

array([3])

In [106]:
X_inputttt[0]

array([[9.41593044e-07, 1.00000000e+00]])

In [107]:
y_output[0]

1.5667942e-05

In [15]:
model.predict([[9.41593044e-07,1]])

1/1 [==============================] - 0s 176ms/step


array([[-0.00108859]], dtype=float32)

In [20]:
save_model(model, 'asd.json')

In [129]:
model = keras.Sequential(name='model')
model.add(keras.layers.InputLayer(input_shape=(None,1,2)))
model.add(keras.layers.LSTM(16, name='L'))
model.add(keras.layers.Dense(16,name='D3'))
model.add(keras.layers.Dense(1, name='fc'))

In [158]:
model.predict([[X_inputtt[0]]])

ValueError: in user code:

    File "/Users/cooky/miniforge3/lib/python3.10/site-packages/keras/engine/training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "/Users/cooky/miniforge3/lib/python3.10/site-packages/keras/engine/training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/cooky/miniforge3/lib/python3.10/site-packages/keras/engine/training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/cooky/miniforge3/lib/python3.10/site-packages/keras/engine/training.py", line 2111, in predict_step
        return self(x, training=False)
    File "/Users/cooky/miniforge3/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/cooky/miniforge3/lib/python3.10/site-packages/keras/engine/input_spec.py", line 235, in assert_input_

In [159]:
X_inputtt[0]

array([7.06194783e-07, 1.00000000e+00])